<a href="https://colab.research.google.com/github/DINESHGITAYE/Digit-Recognizer/blob/main/VGG16_and_ResNet50.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import keras
from keras import layers
from keras.models import Model, Sequential
from keras.layers import Dense, Conv2D, Flatten, MaxPool2D
from keras.utils import to_categorical
from keras.datasets import fashion_mnist


from keras.applications.vgg16 import VGG16
%matplotlib inline

In [ ]:
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [ ]:
X_train.shape

(60000, 28, 28)

# Image set up:
- 1. Convert images into 3 channels.
- 2. Reshape the image as per the tensor format.
- 3. Reshape the image into 48 * 48. This is a requiremet of VGG16.
- 4. Normalize the image data and change data type to float.

## Step 1:

In [ ]:
X_train = np.dstack([X_train] * 3)
X_test = np.dstack([X_test] * 3)

## Step 2:

In [ ]:
X_train = X_train.reshape(-1, 28,28,3)
X_test= X_test.reshape (-1,28,28,3)
X_train.shape, X_test.shape

((60000, 28, 28, 3), (10000, 28, 28, 3))

## Step 3:

In [ ]:
from keras.preprocessing.image import img_to_array, array_to_img

X_train = np.asarray([img_to_array(array_to_img(im, scale=False).resize((48,48))) for im in X_train])
X_test = np.asarray([img_to_array(array_to_img(im, scale=False).resize((48,48))) for im in X_test])
#train_x = preprocess_input(x)
X_train.shape, X_test.shape

((60000, 48, 48, 3), (10000, 48, 48, 3))

## Step 4:

In [ ]:
# Normalise the data and change data type
X_train = X_train / 255.
X_test = X_test/ 255.
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train.shape, X_test.shape

((60000, 48, 48, 3), (10000, 48, 48, 3))

In [ ]:
# Converting Labels to one hot encoded format
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

# VGG16

In [ ]:
import keras
from keras import models
from keras import layers
from keras import optimizers
from keras.applications.vgg16 import VGG16
from keras.layers import Activation, Dense

In [ ]:
# Define the parameters for the VGG16 model
IMG_WIDTH = 48
IMG_HEIGHT = 48
IMG_DEPTH = 3
BATCH_SIZE = 16

In [ ]:
#This is the setup for the VGG16 network:

model_vgg16= VGG16( 
                  input_shape=(IMG_HEIGHT, IMG_WIDTH, IMG_DEPTH),
                   weights = 'imagenet',
                   include_top = False
                 )

model_vgg16.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 48, 48, 3)]       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 48, 48, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 48, 48, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 24, 24, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 24, 24, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 24, 24, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 12, 12, 128)       0     

In [ ]:
for layer in model_vgg16.layers[:-4]:
    layer.trainable = False

The above code stops the model to trained again. 
we are doing this because the vgg16 is already trained model on lots of images.

In [ ]:
model= Sequential()

model.add(model_vgg16)

model.add(layers.Flatten())
model.add(layers.Dense(100, activation='relu'))
#model.add(layers.Dense(50, activation = 'relu'))
#model.add(layers.Dropout(0.5))
model.add(layers.Dense(10, activation='softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 1, 1, 512)         14714688  
_________________________________________________________________
flatten (Flatten)            (None, 512)               0         
_________________________________________________________________
dense (Dense)                (None, 100)               51300     
_________________________________________________________________
dense_1 (Dense)              (None, 50)                5050      
_________________________________________________________________
dense_2 (Dense)              (None, 10)                510       
Total params: 14,771,548
Trainable params: 7,136,284
Non-trainable params: 7,635,264
_________________________________________________________________


In [ ]:
model.compile(
    loss='categorical_crossentropy',
    optimizer= 'adam',
    metrics=['acc'])

In [ ]:
model.fit(X_train, y_train,
          batch_size=128,
          epochs=15,
          verbose=1,
          validation_data=(X_test, y_test))

Epoch 1/15
469/469 [==============================] - 13s 28ms/step - loss: 0.5434 - acc: 0.8009 - val_loss: 0.3869 - val_acc: 0.8591
Epoch 2/15
469/469 [==============================] - 13s 27ms/step - loss: 0.3464 - acc: 0.8720 - val_loss: 0.3429 - val_acc: 0.8786
Epoch 3/15
469/469 [==============================] - 13s 27ms/step - loss: 0.3094 - acc: 0.8854 - val_loss: 0.3231 - val_acc: 0.8805
Epoch 4/15
469/469 [==============================] - 13s 27ms/step - loss: 0.2853 - acc: 0.8946 - val_loss: 0.3501 - val_acc: 0.8747
Epoch 5/15
469/469 [==============================] - 12s 26ms/step - loss: 0.2663 - acc: 0.9010 - val_loss: 0.3233 - val_acc: 0.8886
Epoch 6/15
469/469 [==============================] - 12s 26ms/step - loss: 0.2448 - acc: 0.9087 - val_loss: 0.3103 - val_acc: 0.8907
Epoch 7/15
469/469 [==============================] - 12s 26ms/step - loss: 0.2326 - acc: 0.9147 - val_loss: 0.3325 - val_acc: 0.8834
Epoch 8/15
469/469 [==============================] - 12s 26ms

# ResNet50:

In [ ]:
from keras.applications.resnet50 import ResNet50

In [ ]:
model_resnet50= ResNet50( 
                  input_shape=(IMG_HEIGHT, IMG_WIDTH, IMG_DEPTH),
                   weights = 'imagenet',
                   include_top = False
                 )

model_resnet50.summary()

94773248/94765736 [==============================] - 1s 0us/step
Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 48, 48, 3)]  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 54, 54, 3)    0           input_3[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 24, 24, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 24, 24, 64)   256         conv1_conv[0][0]                 
__________________________

In [ ]:
for layer in model_resnet50.layers[:-4]:
    layer.trainable = False

In [ ]:
model_1= Sequential()

model_1.add(model_resnet50)

model_1.add(layers.Flatten())
model_1.add(layers.Dense(100, activation='relu'))
model_1.add(layers.Dropout(0.5))
model_1.add(layers.Dense(10, activation='softmax'))

model_1.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Functional)        (None, 2, 2, 2048)        23587712  
_________________________________________________________________
flatten_2 (Flatten)          (None, 8192)              0         
_________________________________________________________________
dense_5 (Dense)              (None, 100)               819300    
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 10)                1010      
Total params: 24,408,022
Trainable params: 1,875,030
Non-trainable params: 22,532,992
_________________________________________________________________


In [ ]:
model_1.compile(
    loss='categorical_crossentropy',
    optimizer= 'adam',
    metrics=['accuracy'])

In [ ]:
model_1.fit(X_train, y_train,
          batch_size=128,
          epochs=15,
          verbose=1,
          validation_data=(X_test, y_test))

Epoch 1/15
469/469 [==============================] - 13s 29ms/step - loss: 0.9643 - accuracy: 0.6495 - val_loss: 0.7949 - val_accuracy: 0.7116
Epoch 2/15
469/469 [==============================] - 12s 27ms/step - loss: 0.7474 - accuracy: 0.7291 - val_loss: 0.9577 - val_accuracy: 0.6972
Epoch 3/15
469/469 [==============================] - 12s 27ms/step - loss: 0.6778 - accuracy: 0.7547 - val_loss: 0.6611 - val_accuracy: 0.7515
Epoch 4/15
469/469 [==============================] - 12s 27ms/step - loss: 0.6366 - accuracy: 0.7714 - val_loss: 0.6958 - val_accuracy: 0.7512
Epoch 5/15
469/469 [==============================] - 12s 27ms/step - loss: 0.6152 - accuracy: 0.7786 - val_loss: 0.6402 - val_accuracy: 0.7585
Epoch 6/15
469/469 [==============================] - 12s 27ms/step - loss: 0.5974 - accuracy: 0.7862 - val_loss: 0.6067 - val_accuracy: 0.7759
Epoch 7/15
469/469 [==============================] - 12s 27ms/step - loss: 0.5760 - accuracy: 0.7928 - val_loss: 0.6117 - val_accuracy: